In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
tf.__version__

'0.12.1'

In [4]:
def init_weights(shape):
    w = tf.Variable(tf.random_normal(shape, stddev=0.01))
    return(w)

#P_ are for dropout keep probab
#Inp -> H1 -> H2 -> Outp
#Notice order inside matmul

def model(X, w_ih1, w_h1h2, w_h2o, p_ih1, p_h1h2, p_h2o):
    
    #X is input
    X = tf.nn.dropout(X, p_ih1)
    
    h1 = tf.nn.relu(tf.matmul(X, w_ih1))
    h1 = tf.nn.dropout(h1, p_h1h2)
    
    h2 = tf.nn.relu(tf.matmul(h1, w_h1h2))
    h2 = tf.nn.dropout(h2, p_h2o)
    
    o = tf.matmul(h2, w_h2o)
    
    return(o)


mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

tr_X = mnist.train.images
tr_y = mnist.train.labels

te_X = mnist.test.images
te_y = mnist.test.labels

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [6]:
X = tf.placeholder("float", [None, 784])
y = tf.placeholder("float", [None,  10])

w_ih1  = init_weights([784, 512])
w_h1h2 = init_weights([512, 256])
w_h2o  = init_weights([256,  10])

p_ih1  = tf.placeholder("float")
p_h1h2 = tf.placeholder("float")
p_h2o  = tf.placeholder("float")

y_hat = model(X, w_ih1, w_h1h2, w_h2o, p_ih1, p_h1h2, p_h2o)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_hat, labels=y))

train_op = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)
predict_op = tf.argmax(y_hat, axis=1)



In [11]:
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    
#Train in batches    
    for i in range(100):
        print("Training iter ", i)
        start_list = range(0, len(tr_X), 128) #List for all start positions
        end_list = range(128, len(tr_X)+1, 128)#List for all end positions
        
        for start, end in zip(start_list, end_list):
            sess.run(train_op, feed_dict={
                X : tr_X[start:end],
                y : tr_y[start:end],
                p_ih1  : 0.8,
                p_h1h2 : 0.5,
                p_h2o  : 0.5
                
            })
        
        
#Testing

        test_preds = sess.run(predict_op, feed_dict={
                X : te_X,
                y : te_y,
                p_ih1  : 1.0,
                p_h1h2 : 1.0,
                p_h2o  : 1.0
        })

        test_acc = np.mean(np.argmax(te_y, axis=1) == test_preds)
        print(i, test_acc)
        
        

Training iter  0
0 0.9251
Training iter  1
1 0.9557
Training iter  2
2 0.9663
Training iter  3
3 0.9702
Training iter  4
4 0.9741
Training iter  5
5 0.9758
Training iter  6
6 0.9767
Training iter  7
7 0.9785
Training iter  8
8 0.9774
Training iter  9
9 0.9779
Training iter  10
10 0.9795
Training iter  11
11 0.979
Training iter  12
12 0.9805
Training iter  13
13 0.98
Training iter  14
14 0.9795
Training iter  15
15 0.9813
Training iter  16
16 0.9798
Training iter  17
17 0.9806
Training iter  18
18 0.9806
Training iter  19
19 0.981
Training iter  20
20 0.9824
Training iter  21
21 0.9815
Training iter  22
22 0.9814
Training iter  23
23 0.9825
Training iter  24
24 0.9815
Training iter  25
25 0.9824
Training iter  26
26 0.9817
Training iter  27
27 0.9826
Training iter  28
28 0.9817
Training iter  29
29 0.9827
Training iter  30
30 0.982
Training iter  31
31 0.9831
Training iter  32
32 0.9831
Training iter  33
33 0.9827
Training iter  34
34 0.9816
Training iter  35
35 0.9825
Training iter  36